In [ ]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [ ]:
mod = SourceModule("""
  #include <stdio.h>
  #include <math.h>

  __global__ void matmul(float *a, float *b, float *c, int *a_shape, int *b_shape)
  {
      if((blockDim.y * blockIdx.y + threadIdx.y) < a_shape[0] && (blockDim.x * blockIdx.x + threadIdx.x) < b_shape[1])
      {
        int aMin = (blockDim.y * blockIdx.y + threadIdx.y) * a_shape[1]; 
        int aMax = (blockDim.y * blockIdx.y + threadIdx.y + 1) *  a_shape[1]; 
        int aStep = 1;
        int bMin = blockDim.x * blockIdx.x + threadIdx.x;
        int bMax = blockDim.x * blockIdx.x + threadIdx.x + b_shape[0]*b_shape[1];
        int bStep = b_shape[1];
        float temp = 0;
        for(int ai=aMin, bi = bMin; ai < aMax && bi < bMax; ai += aStep, bi += bStep)
        {
                temp += a[ai] * b[bi];
        }
        int a_index = (blockDim.y * blockIdx.y + threadIdx.y) * b_shape[1];
        c[a_index+bMin] = temp;
    } 
  }
  __global__ void transpose(float *a, float *a_T, int *a_shape) 
  {
      int elem_idx = (blockDim.y * blockIdx.y + threadIdx.y) * a_shape[1] +  blockDim.x * blockIdx.x + threadIdx.x;
      if (elem_idx < a_shape[0]*a_shape[1]) 
          {
              int a_t_1 = a_shape[0];
              int elem_tr_idx =  (blockDim.x * blockIdx.x + threadIdx.x) * a_t_1 +  blockDim.y * blockIdx.y + threadIdx.y;
              a_T[elem_tr_idx] = a[elem_idx];
          }
  
  }
  
  __global__ void row_mean(float *a, float *mean, int *a_shape)
  {
  //Returns a column
      int row_num = (blockDim.x * blockIdx.x + threadIdx.x);
      if (row_num < a_shape[0])
      {
          int start_idx = row_num*a_shape[1];
          int end_idx = start_idx + a_shape[1];
          float sum = 0;
          for (int i = start_idx; i< end_idx; i++) 
          {
              sum += a[i];
          }
          mean[row_num] = sum/a_shape[1];
      }
  }
  
  __global__ void column_mean(float *a, float *mean, int *a_shape)
  {
  //Returns a row
      int col_num = (blockDim.x * blockIdx.x + threadIdx.x);
      if (col_num < a_shape[1])
      {
          int start_idx = col_num;
          int end_idx = start_idx + a_shape[1]*a_shape[0];
          float sum = 0;
          for (int i = start_idx; i< end_idx; i+= a_shape[1]) 
          {
              sum += a[i];
          }
          mean[col_num] = sum/a_shape[0];
      }
  }
  
  __global__ void min_row(float *a, int *a_shape, float *min_row, int *arg_min)
  {
    //Returns a column for min_row and argmin 
      int row_num = (blockDim.x * blockIdx.x + threadIdx.x);
      if (row_num < a_shape[0])
      {
          int start_idx = row_num*a_shape[1];
          int end_idx = start_idx + a_shape[1];
          min_row[row_num] = a[start_idx];
          arg_min[row_num] = 0;
          for (int col = start_idx+1, index=1; col< end_idx, index < a_shape[1]; col++, index ++) 
          {
              if (a[col] < min_row[row_num])
              {
                  min_row[row_num] = a[col];
                  arg_min[row_num] = index;
              }
          }
      }
  
  }
  
  __global__ void sum_axis3(float *a, int *a_shape, float *result)
  {
      //a[i][j][k] = k+a_shape[2]*j + a_shape[2]*a_shape[1]*i
      
      int col_num = (blockDim.x * blockIdx.x + threadIdx.x);
      int row_num = (blockDim.y * blockIdx.y + threadIdx.y);
      if (row_num < a_shape[0] && col_num < a_shape[1])
      {
          int start_idx =(row_num*a_shape[1] + col_num)*a_shape[2];
          int end_idx = start_idx + a_shape[2];
          int step = 1;
          float temp = 0;
          for (int idx = start_idx; idx < end_idx; idx+= step) 
          {
              temp += a[idx];
          }
          result[row_num*a_shape[1] + col_num] = temp;
      }
  
  }
  
    __global__ void sum_axis2(float *a, int *a_shape, float *result)
  {
      //a[i][j][k] = k+a_shape[2]*j + a_shape[2]*a_shape[1]*i
      
      int col_num = (blockDim.x * blockIdx.x + threadIdx.x);
      int row_num = (blockDim.y * blockIdx.y + threadIdx.y);
      if (row_num < a_shape[0] && col_num < a_shape[2])
      {
          int start_idx =row_num*a_shape[1]*a_shape[2] + col_num;
          int end_idx = start_idx + a_shape[2]*a_shape[1];
          int step = a_shape[2];
          float temp = 0;
          for (int idx = start_idx; idx < end_idx; idx+= step) 
          {
              temp += a[idx];
          }
          result[row_num*a_shape[2] + col_num] = temp;
      }
  
  }
  
    __global__ void sum_axis1(float *a, int *a_shape, float *result)
  {
      //a[i][j][k] = k+a_shape[2]*j + a_shape[2]*a_shape[1]*i
      
      int col_num = (blockDim.x * blockIdx.x + threadIdx.x);
      int row_num = (blockDim.y * blockIdx.y + threadIdx.y);
      if (row_num < a_shape[1] && col_num < a_shape[2])
      {
          int start_idx =(row_num)*a_shape[2] + col_num;
          int end_idx = start_idx + a_shape[2]*a_shape[1]*a_shape[0];
          int step = a_shape[2]*a_shape[1];
          float temp = 0;
          for (int idx = start_idx; idx < end_idx; idx+= step) 
          {
              temp += a[idx];
          }
          result[row_num*a_shape[2] + col_num] = temp;
      }
  
  }
  
  
  """)

In [ ]:
a = np.random.randn(10549, 8982).astype(np.float32)
b = np.random.randn(8982, 10549).astype(np.float32)
c = np.zeros([a.shape[0], b.shape[1]]).astype(np.float32)
SHAPE_A = np.array(a.shape).astype(np.uint32)
SHAPE_B = np.array(b.shape).astype(np.uint32)
print(SHAPE_A, SHAPE_B)

In [ ]:
a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu, a)
b_gpu = cuda.mem_alloc(b.nbytes)
cuda.memcpy_htod(b_gpu, b)
c_gpu = cuda.mem_alloc(c.nbytes)
cuda.memcpy_htod(c_gpu, c)

SHAPE_A_gpu = cuda.mem_alloc(SHAPE_A.nbytes)
cuda.memcpy_htod(SHAPE_A_gpu, SHAPE_A)
SHAPE_B_gpu = cuda.mem_alloc(SHAPE_B.nbytes)
cuda.memcpy_htod(SHAPE_B_gpu, SHAPE_B)

In [ ]:
func = mod.get_function("matmul")

In [ ]:
BLOCK_DIMX = 32
BLOCK_DIMY = 32
GRID_DIMX = int(np.ceil(b.shape[1]/float(BLOCK_DIMX)))
GRID_DIMY = int(np.ceil(a.shape[0]/float(BLOCK_DIMY)))

print (GRID_DIMX, GRID_DIMY)

In [ ]:
%%time
func(a_gpu, b_gpu, c_gpu, SHAPE_A_gpu, SHAPE_B_gpu, block=(BLOCK_DIMX, BLOCK_DIMY, 1), grid=(GRID_DIMX, GRID_DIMY, 1))

In [ ]:
results = np.empty_like(c)
cuda.memcpy_dtoh(results, c_gpu)
print(results)
print(np.allclose(np.matmul(a, b), results, atol=1e-2))

In [ ]:
%%time
np.matmul(a, b)

In [ ]:
##TRANSPOSE##
a = np.random.randn(10549, 8982).astype(np.float32)
a_T = np.zeros(list(reversed(a.shape))).astype(np.float32)
SHAPE_A = np.array(a.shape).astype(np.uint32)

In [ ]:
a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu, a)
a_T_gpu = cuda.mem_alloc(a_T.nbytes)
cuda.memcpy_htod(a_T_gpu, a_T)
SHAPE_A_gpu = cuda.mem_alloc(SHAPE_A.nbytes)
cuda.memcpy_htod(SHAPE_A_gpu, SHAPE_A)

In [ ]:
func = mod.get_function("transpose")
BLOCK_DIMX = 32
BLOCK_DIMY = 32
GRID_DIMX = int(np.ceil(a.shape[1]/float(BLOCK_DIMX)))
GRID_DIMY = int(np.ceil(a.shape[0]/float(BLOCK_DIMY)))
print (GRID_DIMX, GRID_DIMY)



In [ ]:
%%time
func(a_gpu, a_T_gpu, SHAPE_A_gpu, block=(BLOCK_DIMX, BLOCK_DIMY, 1), grid=(GRID_DIMX, GRID_DIMY, 1))

In [ ]:
results = np.empty_like(a_T)
cuda.memcpy_dtoh(results, a_T_gpu)
print(results)
print(np.allclose(np.transpose(a), results, atol=1e-4))

In [ ]:
%%time
np.transpose(a)

In [ ]:
## ROW MEAN ##
a = np.random.randn(10549, 8982).astype(np.float32)
mean_a = np.zeros([a.shape[0]]).astype(np.float32)
SHAPE_A = np.array(a.shape).astype(np.uint32)

In [ ]:
a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu, a)
mean_a_gpu = cuda.mem_alloc(mean_a.nbytes)
cuda.memcpy_htod(mean_a_gpu, mean_a)
SHAPE_A_gpu = cuda.mem_alloc(SHAPE_A.nbytes)
cuda.memcpy_htod(SHAPE_A_gpu, SHAPE_A)

In [ ]:
func = mod.get_function("row_mean")
BLOCK_DIMX = 1024
GRID_DIMX = int(np.ceil(a.shape[1]*a.shape[0]/float(BLOCK_DIMX)))
print (GRID_DIMX)

In [ ]:
%%time
func(a_gpu, mean_a_gpu, SHAPE_A_gpu, block=(BLOCK_DIMX, 1, 1), grid=(GRID_DIMX, 1, 1))

In [ ]:
results = np.empty_like(mean_a)
cuda.memcpy_dtoh(results, mean_a_gpu)
print(results)
print(np.allclose(np.mean(a, axis=1), results, atol=1e-4))

In [ ]:
%%time
np.mean(a, axis=1)

In [ ]:
## COLUMN MEAN ##
a = np.random.randn(10549, 8982).astype(np.float32)
mean_a = np.zeros([a.shape[1]]).astype(np.float32)
SHAPE_A = np.array(a.shape).astype(np.uint32)

In [ ]:
a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu, a)
mean_a_gpu = cuda.mem_alloc(mean_a.nbytes)
cuda.memcpy_htod(mean_a_gpu, mean_a)
SHAPE_A_gpu = cuda.mem_alloc(SHAPE_A.nbytes)
cuda.memcpy_htod(SHAPE_A_gpu, SHAPE_A)

In [ ]:
func = mod.get_function("column_mean")
BLOCK_DIMX = 1024
GRID_DIMX = int(np.ceil(a.shape[1]*a.shape[0]/float(BLOCK_DIMX)))
print (GRID_DIMX)

In [ ]:
%%time
func(a_gpu, mean_a_gpu, SHAPE_A_gpu, block=(BLOCK_DIMX, 1, 1), grid=(GRID_DIMX, 1, 1))

In [ ]:
results = np.empty_like(mean_a)
cuda.memcpy_dtoh(results, mean_a_gpu)
print(results)
print(np.allclose(np.mean(a, axis=0), results, atol=1e-4))

In [ ]:
%%time
np.mean(a, axis=0)

In [ ]:
## ARGMIN, MIN ##
a = np.random.randn(10549, 8982).astype(np.float32)
min_a = np.zeros([a.shape[0]]).astype(np.float32)
argmin_a = np.zeros([a.shape[0]]).astype(np.uint32)
SHAPE_A = np.array(a.shape).astype(np.uint32)

In [ ]:
a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu, a)
min_a_gpu = cuda.mem_alloc(min_a.nbytes)
cuda.memcpy_htod(min_a_gpu, min_a)
argmin_a_gpu = cuda.mem_alloc(argmin_a.nbytes)
cuda.memcpy_htod(argmin_a_gpu, argmin_a)
SHAPE_A_gpu = cuda.mem_alloc(SHAPE_A.nbytes)
cuda.memcpy_htod(SHAPE_A_gpu, SHAPE_A)

In [ ]:
func = mod.get_function("min_row")
BLOCK_DIMX = 1024
GRID_DIMX = int(np.ceil(a.shape[1]*a.shape[0]/float(BLOCK_DIMX)))
print (GRID_DIMX)

In [ ]:
%%time
func(a_gpu, SHAPE_A_gpu, min_a_gpu, argmin_a_gpu, block=(BLOCK_DIMX, 1, 1), grid=(GRID_DIMX, 1, 1))

In [ ]:
results = np.empty_like(min_a)
cuda.memcpy_dtoh(results, min_a_gpu)
print(results)
print(np.allclose(np.min(a, axis=1), results, atol=1e-4))

In [ ]:
%%time
np.min(a, axis=1)

In [ ]:
results = np.empty_like(argmin_a)
cuda.memcpy_dtoh(results, argmin_a_gpu)
print(results)
print(np.allclose(np.argmin(a, axis=1), results, atol=1e-4))

In [ ]:
## SUM AXIS##
a = np.random.randn(1054, 89,45).astype(np.float32)
sum3_a = np.zeros([a.shape[0], a.shape[1]]).astype(np.float32)
sum2_a = np.zeros([a.shape[0], a.shape[2]]).astype(np.float32)
sum1_a = np.zeros([a.shape[1], a.shape[2]]).astype(np.float32)
SHAPE_A = np.array(a.shape).astype(np.uint32)

In [ ]:
a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu, a)

sum3_a_gpu = cuda.mem_alloc(sum3_a.nbytes)
cuda.memcpy_htod(sum3_a_gpu, sum3_a)
sum2_a_gpu = cuda.mem_alloc(sum2_a.nbytes)
cuda.memcpy_htod(sum2_a_gpu, sum2_a)
sum1_a_gpu = cuda.mem_alloc(sum1_a.nbytes)
cuda.memcpy_htod(sum1_a_gpu, sum1_a)

SHAPE_A_gpu = cuda.mem_alloc(SHAPE_A.nbytes)
cuda.memcpy_htod(SHAPE_A_gpu, SHAPE_A)

In [ ]:
func = mod.get_function("sum_axis3")
BLOCK_DIMX = 32
BLOCK_DIMY = 32
GRID_DIMX = int(np.ceil(a.shape[1]/float(BLOCK_DIMX)))
GRID_DIMY = int(np.ceil(a.shape[0]/float(BLOCK_DIMY)))
print (GRID_DIMX, GRID_DIMY)

In [ ]:
%%time
func(a_gpu, SHAPE_A_gpu, sum3_a_gpu, block=(BLOCK_DIMX, BLOCK_DIMY, 1), grid=(GRID_DIMX, GRID_DIMY, 1))

In [ ]:
results = np.empty_like(sum3_a)
cuda.memcpy_dtoh(results, sum3_a_gpu)
print(results)
print(np.allclose(np.sum(a, axis=2), results, atol=1e-5))

In [ ]:
%%time
np.sum(a, axis=2)

In [ ]:
func = mod.get_function("sum_axis2")
BLOCK_DIMX = 32
BLOCK_DIMY = 32
GRID_DIMX = int(np.ceil(a.shape[2]/float(BLOCK_DIMX)))
GRID_DIMY = int(np.ceil(a.shape[0]/float(BLOCK_DIMY)))
print (GRID_DIMX, GRID_DIMY)

In [ ]:
%%time
func(a_gpu, SHAPE_A_gpu, sum2_a_gpu, block=(BLOCK_DIMX, BLOCK_DIMY, 1), grid=(GRID_DIMX, GRID_DIMY, 1))

In [ ]:
results = np.empty_like(sum2_a)
cuda.memcpy_dtoh(results, sum2_a_gpu)
print(results)
print(np.allclose(np.sum(a, axis=1), results, atol=1e-5))

In [ ]:
%%time
np.sum(a, axis=1)

In [ ]:
func = mod.get_function("sum_axis1")
BLOCK_DIMX = 32
BLOCK_DIMY = 32
GRID_DIMX = int(np.ceil(a.shape[2]/float(BLOCK_DIMX)))
GRID_DIMY = int(np.ceil(a.shape[1]/float(BLOCK_DIMY)))
print (GRID_DIMX, GRID_DIMY)

In [ ]:
%%time
func(a_gpu, SHAPE_A_gpu, sum1_a_gpu, block=(BLOCK_DIMX, BLOCK_DIMY, 1), grid=(GRID_DIMX, GRID_DIMY, 1))

In [ ]:
results = np.empty_like(sum1_a)
cuda.memcpy_dtoh(results, sum1_a_gpu)
print(results)
print(np.allclose(np.sum(a, axis=0), results, atol=1e-5))

In [ ]:
%%time
np.sum(a, axis=0)